In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## Clasificacion: canjeo de coupones de descuento

Trabajaremos con el dataset preprocesado en la notebook 'opcional_preprocesamiento_datos_clase'.
El mismo cuenta con información de distintas campañas de marketing en donde se ofrecen coupnes de descuento sobre distintas marcas y productos. El dataset preprocesado cuenta con fetures con información sobre las campañas, sobre los consumidores (hábitos de consumo y características demográficas) y sobre los items alcanzados por las promociones.

La variable target es el estado de canje de los cupones: "redemption_status". 

El dataset original está [acá](https://www.kaggle.com/vasudeva009/coupon-redemption-smote-feature-selection/data). Se aconseja mirar la notebook de preprocesamiento para entender mejor las variables del dataset.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTENC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif,RFECV,RFE
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,plot_confusion_matrix,roc_auc_score, classification_report, confusion_matrix, precision_recall_curve, auc

In [ ]:
data=pd.read_csv('../Data/marketing/data_preprocessed.csv')
display(data.head(3))
data.dtypes

In [ ]:
data=data.drop(['Unnamed: 0','id','customer_id','campaign_id','coupon_id'],axis=1)

In [ ]:
categorical=['campaign_type','month','year','age_range','marital_status','rented','family_size','no_of_children',\
             'income_bracket','brand','brand_type','category']

data[categorical]=data[categorical].astype('category')
data.dtypes

### Primera parte
* Ver el balance de clases en el dataset conun value counts de la columna redemption_status
* Hacer un train-test-split estratificado
* Convertir las variables categoricas a dummy. Conviene que nos quedemos con una versión no dummy del dataset de entrenamiento puesto que luego usaremos SMOTENC
* Opcional (recomendado): Armar una función que tome como input un modelo entrenado y un dataset de testeo e imprima las métricas más importantes para evaluar clasificación (classification_report, matriz de confución, area bajo las curvas ROC y Precision-Recall)
* Instanciar un modelo Random Forest entrenarlo y evaluarlo en el dataset de testeo

In [ ]:
data['redemption_status'].value_counts(normalize=True)

In [ ]:
# Train test split

X_train,X_test,y_train,y_test=train_test_split(data.drop('redemption_status',axis=1),data['redemption_status'],\
                                               stratify=data['redemption_status'],random_state=0)


In [ ]:
X_train_dummy=pd.get_dummies(X_train);
X_test_dummy=pd.get_dummies(X_test)
print('Mismas categorías en Train y Test:',(X_train_dummy.columns==X_test_dummy.columns).all())

In [ ]:
def evaluate_model(model,X,y_true):
    '''
    Calcula las métricas ppales para evaluar un clasificador
    Toma como imput el modelo entrenado, el dataset de testeo y sus etiquetas
    '''
    y_pred=model.predict(X)
    y_proba=model.predict_proba(X)

    print(classification_report(y_true,y_pred))
    print('Area bajo la curva ROC:',np.round(roc_auc_score(y_true,y_proba[:,1]),4))
    precision, recall,threshold=precision_recall_curve(y_true,y_proba[:,1]);
    print('Area bajo la curva Precision-Recall:',np.round(auc(recall,precision),4))
    plot_confusion_matrix(model,X,y_true,cmap='Blues');
    return

In [ ]:
model=RandomForestClassifier()
model.fit(X_train_dummy,y_train)
evaluate_model(model,X_test_dummy,y_test)

### Resampling de las clases

* Hacer un undersampling de la clase mayoritaria en el training set para balancear las clases. ¿De qué tamaño quedó el dataset de entrenamiento?
* Volver a entrenar un random forest y evaluarlo en test
* Repetir el procedimiento usando SMOTENC para sobresamplear la clase minoritaria
* Instanciar un Random Forest usando class_weight='balanced_subsample', entrenarlo en el training set original (sin resampling) y evaluarlo en test set

* Combinar las tres estrategias anteriores:
- Undersampling
- Oversampling
- Class weight
Usar una combinación de undersampling y oversampling que les parezca razonable. Idealmente habría que optimizar esta combinación mediante cross-validation, pero no hay que subestimar el tiempo de cómputo. Utilicen una combinación de sampling_stategy que de por resultado un dataset de tamaño reducido respecto del original. Usaremos ese dataset para hacer feature selection en la segunda parte

In [ ]:
# UnderSampling del dataset

sampler=RandomUnderSampler()
X_train_us,y_train_us=sampler.fit_resample(X_train_dummy,y_train)
print('X_train_us:',X_train_us.shape)
print('\nBalance de clases en train:')
print(y_train_us.value_counts())

print('\n\nX_test:',X_test_dummy.shape)
print('\nBalance de clases en test:')
print(y_test.value_counts())




In [ ]:
model=RandomForestClassifier()
model.fit(X_train_us,y_train_us)
print('DATASET SUBSAMPLEADO')
evaluate_model(model,X_test_dummy,y_test)

In [ ]:
print('DATASET SOBRESAMPLEADO')
categorical_mask=(X_train.dtypes=='category').values
sm=SMOTENC(categorical_features=categorical_mask,sampling_strategy='minority')
X_train_os,y_train_os=sm.fit_resample(X_train,y_train)
X_train_os=pd.get_dummies(X_train_os)

print('X_train_os:',X_train_os.shape)
print('\nBalance de clases en train:')
print(y_train_os.value_counts())

print('\n\nX_test:',X_test_dummy.shape)
print('\nBalance de clases en test:')
print(y_test.value_counts())

In [ ]:
model=RandomForestClassifier()
model.fit(X_train_os,y_train_os)
evaluate_model(model,X_test_dummy,y_test)

In [ ]:
# Balance de clases
model=RandomForestClassifier(class_weight='balanced_subsample')
model.fit(X_train_dummy,y_train)
evaluate_model(model,X_test_dummy,y_test)

In [ ]:
# Combinando Estrategias
sampler=RandomUnderSampler(sampling_strategy=0.05)
X,y=sampler.fit_resample(X_train,y_train)
sm=SMOTENC(categorical_mask,sampling_strategy=0.2)
X_train_rs,y_train_rs=sm.fit_resample( X,y)

X_train_rs=pd.get_dummies(X_train_rs)
if not (X_test_dummy.columns==X_train_rs.columns).all():
    print('Train y Test tienen distintas Categorias:')
    print('Usar OneHotEncoding')

print(y_train_rs.shape)
print(y_train_rs.mean())
    


In [ ]:
# Balance de clases
model=RandomForestClassifier(class_weight='balanced_subsample')
model.fit(X_train_rs,y_train_rs)
evaluate_model(model,X_test_dummy,y_test)

### Segunda Parte: Selección de Features

#### SelectKBest

Primero seleccionemos featuers usando SelectKbest tomando como medida de score el criterio 'f_classif'

* Hacer un pipeline que concatene el selector de features y un RandomForestClassifier con class_weight='balanced_subsample'
* Hacer una gridsearchCV variando el parámetro k del selector de features. Usar scoring='f1'.
* Graficar los resultados del procedimiento de cross-validation: scores de clasificacion vs nro de features
* ¿Con cuántas features se quedarían? ¿Cuáles?
* ¿Cuál es el score en el test set al seleccionar el subset de features elegido?
* Graficar la importancia de las features para el modelo (atributo feature_importance_)

In [ ]:
# Seleccion de features:
skf=StratifiedKFold(n_splits=3,shuffle=True,random_state=0)
steps=([('selector',SelectKBest(f_classif)),('classif',RandomForestClassifier(class_weight='balanced_subsample'))])
pipe=Pipeline(steps)
param_grid={'selector__k':np.arange(10,150,20)}
grid=GridSearchCV(pipe,param_grid,scoring='f1',cv=skf,verbose=3,n_jobs=3)
grid.fit(X_train_rs,y_train_rs)

In [ ]:
n_features=grid.cv_results_['param_selector__k'].data
mn_cv_score=grid.cv_results_['mean_test_score']
err=grid.cv_results_['std_test_score']
plt.bar(n_features,mn_cv_score,color = "r",width=3,yerr=err,align = "center")
plt.xlabel('Número de features')
plt.ylabel('test score');

In [ ]:
# Evaluamos en test
skb=SelectKBest(f_classif,k=70)
X_train_reduced=skb.fit_transform(X_train_rs,y_train_rs)
X_test_reduced=X_test_dummy.loc[:,skb.get_support()]
model=RandomForestClassifier(class_weight='balanced_subsample')
model.fit(X_train_reduced,y_train_rs)
evaluate_model(model,X_test_reduced,y_test)

In [ ]:
y=np.sort(model.feature_importances_)
x=np.argsort(model.feature_importances_)
x=x[::-1]
feat_names=X_train_rs.columns[skb.get_support()]
labels=feat_names[x]
y=y[::-1]

plt.figure(figsize=(15,8))
plt.bar(range(len(y)),y,color = "r",width=3,align = "center")
plt.xticks(range(len(y)), labels, rotation=90)

plt.xlim([0,30])

#### Seleccion de Features por RFE
* Implementar RFECV usando como modelo un randomforest con clases pesadas. Usar un step grande (50) para tener una primera aproximacion
* Graficar los scores obtenidos para cada numero de features
* Usar RFE para seleccionar el número de features más adecuado. En este caso implementar un paso más fino (step=10), entrenando en todo el dataset de entremiento (sin cross-validation)
* Evaluar el modelo en el test set y mirar las feature_importance_

In [ ]:
# Recursive feature elimination
skf=StratifiedKFold(n_splits=3,shuffle=True,random_state=0)
rfecv = RFECV(RandomForestClassifier(class_weight='balanced_subsample'), step = 50, cv=skf, scoring = 'f1', verbose=2)
rfecv.fit(X_train_rs,y_train_rs)

In [ ]:
#n_features=grid.cv_results_['param_selector__k'].data
mn_cv_score=rfecv.grid_scores_
n_features=np.arange(7,357+50,50)
n_features=np.insert(n_features,0,1)

plt.bar(n_features,mn_cv_score,color = "r",width=3,align = "center")
plt.xlabel('Número de features');
plt.ylabel('test score');

In [ ]:
rfe=RFE(RandomForestClassifier(class_weight='balanced_subsample'),\
       n_features_to_select=70,step=10,verbose=1)

In [ ]:
rfe.fit(X_train_rs,y_train_rs)

In [ ]:
X_train_reduced=X_train_rs.loc[:,rfe.support_]
X_test_reduced=X_test_dummy.loc[:,rfe.support_]
model=RandomForestClassifier(class_weight='balanced_subsample')
model.fit(X_train_reduced,y_train_rs)
evaluate_model(model,X_test_reduced,y_test)

In [ ]:
y=np.sort(model.feature_importances_)
x=np.argsort(model.feature_importances_)
x=x[::-1]
feat_names=X_train_rs.columns[skb.get_support()]
labels=feat_names[x]
y=y[::-1]

plt.figure(figsize=(15,8))
plt.bar(range(len(y)),y,color = "r",width=3,align = "center")
plt.xticks(range(len(y)),labels,rotation=90);
plt.xlim([0,30])